In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = "/"
ADMIN_URL = "http://data-agent:8021"

In [4]:
async def start_agent():
    # Based on the aca-py agent you wish to control
    agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)
    
    def data_messages_hook(payload):
        connection_id = payload["connection_id"]
        print("Handle data messages ", payload, connection_id)


    data_message_listener = {
        "handler": data_messages_hook,
        "topic": "basicmessages"
    }
    
    
    await agent_controller.listen_webhooks()
    
    # Controllers re-emit webhooks as events through PyPubSub
    # Basic state management is handled through defaults but you can choose to pass in custom handlers
    agent_controller.register_listeners([data_message_listener], defaults=True)
    
    # Create Invitation
    invite = await agent_controller.connections.create_invitation(alias="Will")
    print(invite)
    
    await asyncio.sleep(30)
    
    # Print out accepted Invite and Researcher ID
    print("Data Owner ID", invite["connection_id"])
    data_connection_id = invite["connection_id"]
    
    
    # Accept Request for Invite created
#     connection = await agent_controller.connections.accept_request(data_connection_id)
    connection = await agent_controller.connections.admin_POST(f"/connections/{data_connection_id}/accept-request")
    print("ACCEPT REQUEST")
    print(connection)
    
    
    # Do a trust ping to bring accepted request into "ACTIVE" state
    while connection["state"] != "active":
        trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
        print("TUST PING TO ACTIVATE CONNECTION - DATA -> RESEARCH")
        print(trust_ping)
        time.sleep(5)
        connection = await agent_controller.connections.get_connection(data_connection_id)
    
    
    # Print connection list
    connection = await agent_controller.connections.get_connection(data_connection_id)
    print("RESEARCH AGENT CONNECTION")
    print(connection)

    #send some basic messages
    message = await agent_controller.connections.send_message(data_connection_id,"hello from data owner world!")
    print("BASIC MESSAGE - DATA -> RESEARCH")
    print(message)

In [5]:
async def main():
    await asyncio.gather(start_agent())

s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")

{'connection_id': '95e91f5d-3b56-4d36-9a74-ee17568d7b17', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5dde53f0-fd0e-4467-813f-a29da9e3994f', 'recipientKeys': ['8WaYDnM4YYP8ECakf5dTVNbZQuvHTNgVeuZRosHru8Uq'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Bob'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNWRkZTUzZjAtZmQwZS00NDY3LTgxM2YtYTI5ZGE5ZTM5OTRmIiwgInJlY2lwaWVudEtleXMiOiBbIjhXYVlEbk00WVlQOEVDYWtmNWRUVk5iWlF1dkhUTmdWZXVaUm9zSHJ1OFVxIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE5Mi4xNjguNjUuMzo4MDIwIiwgImxhYmVsIjogIkJvYiJ9'}
Data Owner ID 95e91f5d-3b56-4d36-9a74-ee17568d7b17
ACCEPT REQUEST
{'their_did': 'UwKxBXxdvWG8pqBzqBDZTy', 'invitation_mode': 'once', 'updated_at': '2020-06-24 21:31:32.172738Z', 'connection_id': '95e91f5d-3b56-4d36-9a74-ee17568d7b17', 'their_label': 'Alice', 'initiator': 'self', 'routin

In [ ]:
# if __name__ == "__main__":

#     try:
#         asyncio.get_event_loop().run_until_complete(start_agent())
#     except KeyboardInterrupt:
#         os._exit(1)